In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
mon_lst = []
for y in range(2000, 2025):
    for m in ['01-31', '02-29' if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30', '12-31']:
        mon_lst.append(f'{y}-{m}')
mon_df = pd.DataFrame(mon_lst[:-2], columns=['交易月份'])
mon_df['月份'] = mon_df['交易月份'].apply(lambda x: x[:7])
stk_df = pd.read_csv('公司文件.csv')
stk_df['on'] = 0
mon_df['on'] = 0
header = pd.merge(stk_df, mon_df, on=['on'], how='left')[['证券代码', '交易月份', '月份']]
header

,证券代码,交易月份,月份
0,1,2000-01-31,2000-01
1,1,2000-02-29,2000-02
2,1,2000-03-31,2000-03
3,1,2000-04-30,2000-04
4,1,2000-05-31,2000-05
...,...,...,...
1719157,920118,2024-06-30,2024-06
1719158,920118,2024-07-31,2024-07
1719159,920118,2024-08-31,2024-08
1719160,920118,2024-09-30,2024-09


In [5]:
# 15. chcsho
# 月频率。流通股每月变动百分比。

df1 = pd.read_csv('日_股本变动文件.csv', usecols=['证券代码', '股本变动日期', 'A股流通股数', 'B股流通股数', 'H股流通股数'])
df1['月份'] = df1['股本变动日期'].apply(lambda x: x[:7])
df1['流通股数'] = df1['A股流通股数'] + df1['B股流通股数'] + df1['H股流通股数']
df2 = df1.sort_values(by=['证券代码', '股本变动日期'])
df3 = df2.drop_duplicates(subset=['证券代码', '月份'], keep='last')
df3['chcsho'] = (df3['流通股数'] - df3.groupby('证券代码').shift(1)['流通股数'])/df3.groupby('证券代码').shift(1)['流通股数']
df4 = df3[['证券代码', '月份', 'chcsho']]
df5 = pd.merge(header, df4, on=['证券代码', '月份'], how='left')
df6 = pd.read_csv('月_月个股回报率文件.csv', usecols=['证券代码', '交易月份', '查询成功'])
df7 = pd.merge(df5, df6, on=['证券代码', '交易月份'], how='left')
df7['chcsho'][df7['查询成功'] == 1] = df7['chcsho'][df7['查询成功'] == 1].fillna(0)
df8 = df7[['证券代码', '交易月份', 'chcsho']]
df8.to_csv('月_15.csv', index=False, encoding='utf-8-sig')
df8

,证券代码,交易月份,chcsho
0,1,2000-01-31,0.000000
1,1,2000-02-29,0.000000
2,1,2000-03-31,0.000000
3,1,2000-04-30,0.000000
4,1,2000-05-31,0.000000
...,...,...,...
1719157,920118,2024-06-30,NaN
1719158,920118,2024-07-31,NaN
1719159,920118,2024-08-31,0.000000
1719160,920118,2024-09-30,-0.076203
